In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("FakeNewsNet.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23196 entries, 0 to 23195
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          23196 non-null  object
 1   news_url       22866 non-null  object
 2   source_domain  22866 non-null  object
 3   tweet_num      23196 non-null  int64 
 4   real           23196 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 906.2+ KB


In [5]:
df.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [7]:
df=df.drop("news_url",axis=1)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23196 entries, 0 to 23195
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          23196 non-null  object
 1   source_domain  22866 non-null  object
 2   tweet_num      23196 non-null  int64 
 3   real           23196 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 725.0+ KB


In [10]:
df=df.dropna()

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22866 entries, 0 to 23195
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          22866 non-null  object
 1   source_domain  22866 non-null  object
 2   tweet_num      22866 non-null  int64 
 3   real           22866 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 893.2+ KB


In [14]:
df["source_domain"].unique().shape

(2441,)

In [28]:
from sklearn.model_selection import train_test_split
x,xt = train_test_split(df,test_size=0.2,random_state=42)

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,FunctionTransformer

In [18]:
text_cols = ["title","source_domain"]
num_cols = ["tweet_num"]

In [85]:
text_pipeline = Pipeline([
    ("combine", FunctionTransformer(
        lambda x: x.astype(str).agg(" ".join, axis=1),
        validate=False
    )),
    ("tfidf", TfidfVectorizer(max_features=10000))
])


In [86]:
preprocessor = ColumnTransformer(
    transformers=[
        ("text", text_pipeline, text_cols),
        ("num", StandardScaler(), num_cols)
    ]
)


In [87]:
from sklearn.linear_model import LogisticRegression

In [88]:
pipeline = Pipeline([
    ("preprocess", preprocessor),
    ("model", LogisticRegression(max_iter = 1000, penalty ='l2', class_weight='balanced'))
])

In [89]:
x_train = x.drop("real",axis=1)
y_train = x["real"]

In [90]:
x_test = xt.drop("real",axis=1)
y_test = xt["real"]

In [91]:
x_train.shape,y_train.shape

((18292, 3), (18292,))

In [92]:
pipeline.fit(x_train,y_train)

,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('text', ...), ('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [93]:
pipeline.score(x_train,y_train)

0.9037284058604854

In [94]:
pipeline.score(x_test,y_test)

0.854831657192829